### The Cell below imports the functions and packages needed for ThellierCoolPy to run

In [ ]:
import ThellierCoolFunc as tf
from tkinter import Tk
from tkinter.filedialog import askdirectory

### Select your chosen FORC file using the cell below

In [ ]:
fc = tf.FORCfile()

In [ ]:
if ('X' in locals()) == False:
    X = {}

X["fn"] = fc.selected
X = tf.proccess_all(X)

### Name the FORC sample

In [ ]:
name = str(input("Sample name of chosen FORC diagram: " ))
X['name'] = name

### Cells below processes the FORC data for the model. This cell takes ~ 1 minute to run 


In [ ]:
X = tf.prod_FORCs(X)

### Run the two cells below to find FWHM to calcualte SF=0 FORC diagram to use in model

In [ ]:
X = tf.finding_fwhm(X)

In [ ]:
X = tf.plot_fwhm_1(X) 

### This section is checking for anomalous FWHM which should all lie on a line with a slight positive gradient. Remove the outliers by specifing the SF to remove when prompted

In [ ]:
X = tf.check_fwhm(X) 

## Enter the smallest smoothing factor which was accepted during the FWHM section above 

In [ ]:
SF = (input("Input the lowest reliable SF from the FWHM graph above:" )) #make this automatic to be smallest posisble?
X['SF'] = int(SF)

### Plot full FORC diagram of chosen smoothing factor to use in selecting boundaries 


In [ ]:
sf_plot = str(input("Smoothing factor for plotting FORC diagram (between 2 and 5):" ))
X['sf_choose'] = int(sf_plot)
X = tf.divide_mu0(X) #do i need this divide - havent checked in a while
X = tf.sym_norm_forcs(X)
tf.norm_rho_all(X)
tf.plot_sample_FORC(X['Hc'], X['Hu'], X['rho_n'], X['sf_choose'], X['name'])

### Input the lab cooling time, the natural cooling time, the maximum Ms (if not using a Ms-T curve), the Curie temperature (if not using Ms-T curve). Using the FORC diagram above, select the boundaries where the distribution is not zero - to increase model resolution

In [ ]:
lab_cool_time = (input("Input the lab cooling time in hours:" )) #add statements to protect in case of bad input
nature_cool_time = (input("Input the natural cooling time in hours:" ))
X['lab_cool'] = lab_cool_time
X['nat_cool'] = nature_cool_time
curie_t = (input("Input curie temp (if using Ms-T curve input Nan):" ))
max_input = (input("Adjust maximum hc (mT) using FORC diagram:" ))
max_inputhi = (input("Adjust maximum positive hi (mT) using FORC diagram:" ))
X['reset_limit_hc'] = float(max_input)
X['reset_limit_hi'] = float(max_inputhi)
V = {}
V['curie_t'] = float(curie_t)

### Select the location of the folder containing the Thellier plots to correct from the pop up window


In [ ]:
path = askdirectory(title='Select Folder') 

### If using Ms-T curve select it here. Else run the cell but do not select a file 

In [ ]:
ms_t = tf.ms_t_file()

In [ ]:
V, curve = tf.read_file2(ms_t, V)

### This cell will process correct the Thellier plots for the selected cooling rate in the selected folder and save the FORC diagram with the chosen limits - this will take ~ 3 minutes 

In [ ]:
tf.TRM_acq(X, V, curve, path)
V = tf.plots_correction_factor(V,X)
tf.fix_files(path, V)
tf.plot_sample_FORC2(X['Hc'], X['Hu'], X['rho_n'], X['sf_choose'], X['name'],X['reset_limit_hc'],X['reset_limit_hi'])